In [13]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [14]:
df = pd.read_csv('C:/Users/neals/Desktop/Practicum/prompt_results_for_chips.csv')
dataset_dir = 'E:/datasets'
dataset_name = 'DOTA_dataset_512'
coco_dir = f'{dataset_dir}/{dataset_name}'
coco_file = f'{coco_dir}/YOLO_coco_results_annots.json'
with open(coco_file, 'r') as f:
    coco_data = json.load(f)

category_map = {cat['id']:cat['name'] for cat in coco_data['categories']}

In [15]:
df.head()

,image_id,image_file,image_path,prompt,generated_text,1,2,3,4,5,...,7,8,9,10,11,12,13,14,15,16
0,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,A picture of,a parking lot with several yellow school buses,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,A photo of,a parking lot with several yellow school buses,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,There are,several yellow school buses parked in a parkin...,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,The image quality is,"poor, but the image shows a parking lot with s...",0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,P0003_1.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,A picture of,a parking lot with yellow school buses,0,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Tokenize the generated_text column and remove stopwords
df['tokens'] = df['generated_text'].str.lower().str.split()
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in ENGLISH_STOP_WORDS])


In [17]:
prompts = ["A picture of", "A photo of", "There are", "The image quality is"]
word_count_df = None
for prompt in prompts:
    print(f'results for prompt={prompt}')
    filtered_df = df[df['prompt'] == prompt]
    # Create a DataFrame to store word counts for each class
    word_counts_per_class = {}

    # Loop through each class (columns 1 to 16)
    for i in range(1, 17):
        word_counts_per_class[i] = Counter()

        # Loop through each row
        for idx, row in filtered_df.iterrows():
            # If the class is present (assuming non-zero means presence)
            if row[str(i)] > 0:
                # Update the word count for this class
                word_counts_per_class[i].update(row['tokens'])

    # Convert word counts to DataFrame
    class_word_counts = pd.DataFrame(word_counts_per_class).fillna(0)

    # Normalize counts (optional)
    #class_word_counts = class_word_counts.div(class_word_counts.sum(axis=0), axis=1)
    # Find top words for each class based on frequency
    top_words = {}
    top_words_dict_list =[]
    for i in range(1, 17):
        top_words[i] = class_word_counts[i].sort_values(ascending=False).head(10)
        
        top_words_dict_list.append({
            'class_id': i,
            'class_name': category_map[i],
            'prompt': prompt,
            'word_counts': class_word_counts[i].sort_values(ascending=False)
        })
    class_word_counts['prompt'] = prompt
    class_word_counts['word'] = class_word_counts.index
    if word_count_df is None:
        word_count_df = class_word_counts
    else:
        word_count_df = pd.concat([word_count_df,class_word_counts])
    
    # Display top words for each class
    for i in range(1, 17):
        print(f"Class {category_map[i]} top words:\n{top_words[i]}\n")





results for prompt=A picture of
Class small-vehicle top words:
parked      1065.0
lot          893.0
parking      867.0
large        490.0
airport      428.0
building     417.0
field        380.0
cars         361.0
house        345.0
street       322.0
Name: 1, dtype: float64

Class large-vehicle top words:
parked       721.0
lot          644.0
parking      633.0
trucks       339.0
large        313.0
airport      217.0
building     160.0
cars         155.0
warehouse    152.0
tarmac       113.0
Name: 2, dtype: float64

Class ship top words:
water         795.0
boats         677.0
dock          487.0
boat          439.0
marina        427.0
docked        303.0
house         241.0
ship          161.0
parked        141.0
background    110.0
Name: 3, dtype: float64

Class harbor top words:
water         921.0
dock          752.0
boats         617.0
boat          485.0
marina        408.0
house         351.0
docked        284.0
background    155.0
lake          136.0
houses        131.0
Name:

In [18]:
class_word_counts

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,prompt,word
"poor,",3305.0,1297.0,1450.0,1437.0,283.0,353.0,320.0,286.0,235.0,547.0,137.0,795.0,59.0,522.0,500.0,8.0,The image quality is,"poor,"
image,1569.0,821.0,457.0,334.0,107.0,162.0,147.0,77.0,99.0,108.0,67.0,506.0,56.0,115.0,244.0,4.0,The image quality is,image
shows,1144.0,627.0,440.0,341.0,182.0,158.0,216.0,66.0,164.0,83.0,68.0,291.0,37.0,443.0,358.0,4.0,The image quality is,shows
road,229.0,78.0,14.0,7.0,7.0,5.0,10.0,6.0,24.0,1.0,1.0,6.0,5.0,23.0,3.0,0.0,The image quality is,road
yellow,19.0,28.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,The image quality is,yellow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
container,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,The image quality is,container
d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,The image quality is,d
structure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,The image quality is,structure
pipeline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,The image quality is,pipeline


In [19]:
out_file = 'C:/Users/neals/Desktop/Practicum/top_word_counts_for_prompts_on_chips.csv'
word_count_df.to_csv(out_file,index=False)

In [20]:
word_count_df = pd.read_csv(out_file)
word_count_df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,prompt,word
0,282.0,99.0,23.0,11.0,13.0,5.0,11.0,19.0,60.0,3.0,1.0,29.0,5.0,64.0,21.0,0.0,A picture of,road
1,27.0,29.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,A picture of,yellow
2,83.0,47.0,6.0,0.0,13.0,5.0,16.0,5.0,1.0,25.0,8.0,0.0,0.0,0.0,1.0,0.0,A picture of,school
3,28.0,30.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A picture of,bus
4,867.0,633.0,40.0,18.0,8.0,3.0,5.0,12.0,16.0,15.0,6.0,79.0,1.0,4.0,28.0,0.0,A picture of,parking


In [21]:
# Set the 'word' column as the index
word_count_df.set_index('word', inplace=True)


In [22]:
for prompt in list(word_count_df['prompt'].unique()):
    prompt_df = word_count_df[word_count_df['prompt']==prompt]
    select_classes = [2,3,4,6]
    for i in range(1, 17):
        if i in select_classes:
            print(f'{i}:{category_map[i]} | {prompt}')
            print(prompt_df[f'{i}'].sort_values(ascending=False).head(10))

2:large-vehicle | A picture of
word
parked       721.0
lot          644.0
parking      633.0
trucks       339.0
large        313.0
airport      217.0
building     160.0
cars         155.0
warehouse    152.0
tarmac       113.0
Name: 2, dtype: float64
3:ship | A picture of
word
water         795.0
boats         677.0
dock          487.0
boat          439.0
marina        427.0
docked        303.0
house         241.0
ship          161.0
parked        141.0
background    110.0
Name: 3, dtype: float64
4:harbor | A picture of
word
water         921.0
dock          752.0
boats         617.0
boat          485.0
marina        408.0
house         351.0
docked        284.0
background    155.0
lake          136.0
houses        131.0
Name: 4, dtype: float64
6:soccer-ball-field | A picture of
word
field        307.0
soccer       262.0
tennis        63.0
court         60.0
satellite     54.0
city          52.0
image         41.0
baseball      39.0
ball          32.0
park          22.0
Name: 6, dtype: 